In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np

from datetime import datetime, timedelta

import plotly.offline as pl
pl.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn.ensemble import RandomForestClassifier


import parseIntervalFiles as pif
import consolidateFiles as cf
import datacleaning as cl
import dataviz as dv
import fragmentation as fr
import classif_multiclass as cmc
import hervpd as hp

## Pipeline configuration 
* set the input/output directories, users and date range

In [2]:
verbose = True
    
DATA_PATH = "C:\\Users\\ju\\GDrive\\Projects\\HeRV\\Data\\" 
# DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/"
    
RAW_PATH = DATA_PATH + "Raw"
PRE_PATH = DATA_PATH + "PreProcessed"

dt1 = datetime(2018, 6, 18)
dt2 = datetime(2018, 6, 27)

users = [0, 1, 3, 8, 9, 10]

## Extracting sessions

In [3]:
%time sess = cf.gen_sessions_dataset(users, dt1, dt2, dirname=RAW_PATH, verbose=True)

parsing C:\Users\ju\GDrive\Projects\HeRV\Data\Raw\0\act180618.csv
0 sessions excluded
parsing C:\Users\ju\GDrive\Projects\HeRV\Data\Raw\0\act180619.csv
0 sessions excluded
parsing C:\Users\ju\GDrive\Projects\HeRV\Data\Raw\0\act180620.csv
0 sessions excluded
parsing C:\Users\ju\GDrive\Projects\HeRV\Data\Raw\0\act180621.csv
0 sessions excluded
parsing C:\Users\ju\GDrive\Projects\HeRV\Data\Raw\0\act180624.csv
0 sessions excluded
parsing C:\Users\ju\GDrive\Projects\HeRV\Data\Raw\0\act180625.csv
0 sessions excluded
parsing C:\Users\ju\GDrive\Projects\HeRV\Data\Raw\0\act180626.csv
0 sessions excluded
[ 0 ] train-baseline 2018-06-18 11:02:20
[ 0 ] train-focus 2018-06-18 11:04:32
[ 0 ] train-breathe 2018-06-18 11:10:18
[ 0 ] train-focus 2018-06-18 11:14:17
[ 0 ] train-breathe 2018-06-18 11:24:58
[ 0 ] train-focus 2018-06-18 12:35:13
[ 0 ] train-breathe 2018-06-18 12:39:50
[ 0 ] train-baseline 2018-06-18 17:22:39
[ 0 ] train-focus 2018-06-18 17:25:34
[ 0 ] train-breathe 2018-06-18 17:29:15
[ 0 

In [4]:
dfs = pd.DataFrame(sess)

In [5]:
dfs = dfs[dfs.activity.isin(['train-baseline', 'train-focus', 'train-breathe'])]

In [6]:
dfs.groupby('activity').count()['duration']

activity
train-baseline    18
train-breathe     32
train-focus       32
Name: duration, dtype: int64

In [9]:
dfs[dfs.user==10][['activity', 'user', 'start', 'duration', 'beatscount']]

,activity,user,start,duration,beatscount
89,train-baseline,10,2018-06-26 15:49:04,80,89
90,train-focus,10,2018-06-26 15:50:34,80,90
91,train-breathe,10,2018-06-26 15:52:02,80,91
92,train-focus,10,2018-06-26 15:53:25,80,95
93,train-breathe,10,2018-06-26 15:54:56,80,96
94,train-focus,10,2018-06-26 15:56:21,80,96
95,train-breathe,10,2018-06-26 15:58:00,80,96


In [10]:
dfs.describe()

,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,nn50,pnn50,removed_artifacts,rmssd,sdnn,sess_id,total_power,user,vlf
count,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.00000,82.000000,82.000000,82.000000
mean,156.719512,115.682927,1055.037499,32.546377,2231.326394,4.644085,67.453623,80.736224,765.661458,22.353659,16.436727,0.573171,53.171403,69.605656,46.95122,3893.068672,2.743902,606.704779
std,44.521676,27.172462,1670.818867,21.837329,2740.077125,4.882769,21.837329,10.884444,109.595533,17.464866,14.629428,1.515602,34.344753,30.530054,28.88842,3757.292247,3.809705,743.671626
min,79.000000,80.000000,11.699168,4.565933,103.445366,0.217607,17.871682,58.616372,587.802469,0.000000,0.000000,0.000000,6.774447,18.383460,0.00000,229.731041,0.000000,13.689722
25%,121.000000,80.000000,201.460117,12.334949,523.311119,1.099540,52.367346,73.286149,686.891331,8.000000,5.163043,0.000000,27.326873,45.643747,21.25000,1203.431146,0.000000,212.423329
50%,165.000000,128.500000,436.668598,30.966444,1248.789980,2.230101,69.033556,81.319711,747.664889,20.500000,12.266754,0.000000,51.801206,62.217281,47.50000,2254.308665,1.000000,416.511299
75%,188.750000,134.750000,998.486345,47.632654,2686.648190,7.107052,87.665051,88.781364,833.719937,31.000000,24.017217,0.000000,64.796946,89.708790,72.25000,5345.637600,6.750000,613.541847
max,277.000000,171.000000,9310.089652,82.128318,14893.856347,20.901331,95.434067,102.240602,1030.265823,68.000000,65.909091,10.000000,162.705689,161.716777,95.00000,16808.251518,10.000000,3940.988059


In [11]:
dfs.drop(['rr'], axis =  1).to_excel(PRE_PATH + "\\train_sessions.xlsx")


---


# VISUALIZATION

---



In [13]:
#d21 = df_export[df_export.start > '2018-06-21']
for feat in cl.features_all:
    dv.boxplot_compare(dfs[dfs.user==10], feat, groupby='activity', min_examples=1)


---


# FRAGMENTATION

---



In [14]:
sessions = dfs.to_dict(orient='records')

### Example dataset for 60 sec frags

In [15]:
frags = fr.gen_fragments_dataset(sessions, 60, 5, RAW_PATH)
dff = pd.DataFrame(frags)
dff.sample(2)

82 valid sessions out of 82 total (at least one full fragment of 60 seconds after discarding first 5 seconds)


C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 238, using nperseg = 238

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 241, using nperseg = 241

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 234, using nperseg = 234

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 243, using nperseg = 243

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 235, using nperseg = 235

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 246, using nperseg = 246

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spect

,activity,beatcount,f_id,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,rmssd,rr,sdnn,sess_id,start,stop,total_power,user,vlf
121,train-focus,95,0,73.697111,14.262449,443.024183,6.011419,85.737551,94.796957,634.568421,...,sit,10.812227,"[{'date': 2018-06-26 15:38:36, 'interval': 564...",32.052036,85,2018-06-26 15:38:35,2018-06-26 15:39:35,647.224535,9,130.503241
95,train-baseline,61,1,402.606548,11.154406,3206.788252,7.965067,88.845594,61.718704,980.459016,...,sit,51.675913,"[{'date': 2018-06-23 23:38:12, 'interval': 101...",91.946465,57,2018-06-23 23:38:11,2018-06-23 23:39:11,6193.431271,1,2584.036471


In [18]:
dv.full_plot_sess(dfs.iloc[-2], dff, RAW_PATH)

SESSION ID:  94
ACTIVITY:  train-focus
USER:  10
TIME:  2018-06-26 15:56:21


### Generate datasets for multiple fragment sizes

In [19]:
durations = [60, 30, 20, 15, 10]
crop = 10

for dr in durations:
    fname = PRE_PATH + '\\df_train_' + str(dr) + '.xlsx'
    print ('generating', fname, '...')
    ds = pd.DataFrame(fr.gen_fragments_dataset(sessions, dr, crop, RAW_PATH))
    print('resulting dataset:', len(ds), 'records')
    ds.drop('rr', axis=1).to_excel(fname)

generating C:\Users\ju\GDrive\Projects\HeRV\Data\PreProcessed\df_train_60.xlsx ...
82 valid sessions out of 82 total (at least one full fragment of 60 seconds after discarding first 10 seconds)


C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 237, using nperseg = 237

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 239, using nperseg = 239

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 241, using nperseg = 241

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 240, using nperseg = 240

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 238, using nperseg = 238

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 245, using nperseg = 245

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spect

resulting dataset: 122 records
generating C:\Users\ju\GDrive\Projects\HeRV\Data\PreProcessed\df_train_30.xlsx ...
82 valid sessions out of 82 total (at least one full fragment of 30 seconds after discarding first 10 seconds)


C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 116, using nperseg = 116

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 115, using nperseg = 115

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 117, using nperseg = 117

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 118, using nperseg = 118

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 119, using nperseg = 119

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 114, using nperseg = 114

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spect

resulting dataset: 260 records
generating C:\Users\ju\GDrive\Projects\HeRV\Data\PreProcessed\df_train_20.xlsx ...
82 valid sessions out of 82 total (at least one full fragment of 20 seconds after discarding first 10 seconds)


...

resulting dataset: 397 records
generating C:\Users\ju\GDrive\Projects\HeRV\Data\PreProcessed\df_train_15.xlsx ...
82 valid sessions out of 82 total (at least one full fragment of 15 seconds after discarding first 10 seconds)


C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 55, using nperseg = 55

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 56, using nperseg = 56

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 54, using nperseg = 54

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 52, using nperseg = 52

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 51, using nperseg = 51

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 53, using nperseg = 53

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633:

resulting dataset: 535 records
generating C:\Users\ju\GDrive\Projects\HeRV\Data\PreProcessed\df_train_10.xlsx ...
82 valid sessions out of 82 total (at least one full fragment of 10 seconds after discarding first 10 seconds)


C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 33, using nperseg = 33

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 35, using nperseg = 35

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 32, using nperseg = 32

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 38, using nperseg = 38

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 34, using nperseg = 34

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633: UserWarning:

nperseg = 256 is greater than input length  = 36, using nperseg = 36

C:\Users\Ju\Anaconda3\lib\site-packages\scipy\signal\spectral.py:1633:

resulting dataset: 846 records



---


# CLASSIFICATION

---



In [20]:
dff = pd.read_excel(PRE_PATH+'\\df_train_15.xlsx')

In [21]:
dff[dff.user==0].groupby('activity').count()['start']

activity
train-baseline     88
train-breathe     110
train-focus       110
Name: start, dtype: int64

In [23]:
dfr = dff[dff.activity.isin(['train-focus', 'train-breathe'])]

In [24]:
features = cl.features_all
clf1 = RandomForestClassifier(n_estimators=10, n_jobs=-1)
clf2 = RandomForestClassifier(n_estimators=100, n_jobs=-1)
clf3 = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
clf4 = RandomForestClassifier(n_estimators=10000, n_jobs=-1)
clf5 = RandomForestClassifier(n_estimators=100000, n_jobs=-1)

In [28]:
for label in ['activity']:
    cmc.plot_matrices(dfr[dfr.user.isin([9,10])], clf3, features, label)




--------------------------------------------- activity ---------------------------------------------
activity
train-breathe    24
train-focus      24
Name: user, dtype: int64
38 10
